# Using DALI with JAX

This simple example shows how to train a neural network implemented in JAX with DALI pipelines. It builds on MNIST training example from JAX codebse that can be found [here](https://github.com/google/jax/blob/jax-v0.4.13/examples/mnist_classifier_fromscratch.py).

We will use MNIST in Caffe2 format from [DALI_extra](https://github.com/NVIDIA/DALI_extra).

In [1]:
import os

training_data_path = os.path.join(os.environ['DALI_EXTRA_PATH'], 'db/MNIST/training/')
validation_data_path = os.path.join(os.environ['DALI_EXTRA_PATH'], 'db/MNIST/testing/')

First step is to create a pipeline definition function that will later be used to create instances of DALI pipelines. It defines all steps of the preprocessing. In this simple example we have `fn.readers.caffe2` for reading data in Caffe2 format, `fn.decoders.image` for image decoding, `fn.crop_mirror_normalize` used to normalize the images and `fn.reshape` to adjust the shape of the output tensors. We also move the labels from the CPU to the GPU memory with `labels.gpu()` and apply one hot encoding to them for training with `fn.one_hot`.

This example focuses on how to use DALI pipeline with JAX. For more information on DALI pipeline look into [Getting started](../../getting_started.ipynb) and [pipeline documentation](../../../pipeline.rst)

In [2]:
from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
import nvidia.dali.types as types


batch_size = 100
image_size = 28
num_classes = 10


@pipeline_def(device_id=0, batch_size=batch_size, num_threads=4)
def mnist_pipeline(data_path, random_shuffle):
    jpegs, labels = fn.readers.caffe2(
        path=data_path,
        random_shuffle=random_shuffle,
        name="mnist_caffe2_reader")
    images = fn.decoders.image(
        jpegs, device='mixed', output_type=types.GRAY)
    images = fn.crop_mirror_normalize(
        images, dtype=types.FLOAT, std=[255.], output_layout="CHW")
    images = fn.reshape(images, shape=[image_size * image_size])

    labels = labels.gpu()
    
    if random_shuffle:
        labels = fn.one_hot(labels, num_classes=num_classes)

    return images, labels

Next step is to instantiate pipelines and build them. Building creates and initializes pipeline internals.

In [3]:
print('Creating pipelines')
training_pipeline = mnist_pipeline(data_path=training_data_path, random_shuffle=True)
validation_pipeline = mnist_pipeline(data_path=validation_data_path, random_shuffle=False)

print('Building pipelines')
training_pipeline.build()
validation_pipeline.build()

print(training_pipeline)
print(validation_pipeline)

Creating pipelines
Building pipelines


For DALI pipeline to work with JAX it needs to be wrapped with appropriate DALI iterator. To get the iterator compatible with JAX we need to import it from DALI JAX plugin. In addition to the pipeline we can pass the `output_map`, `reader_name` and `auto_reset` parameters to the iterator. 

**Here is a quick explnation of how these parameters work:**

 - `output_map`: iterators return a dictionary with outputs of the pipeline as its values. Keys in this dictionary are defined by `output_map`. For example, `labels` output returned from the DALI pipeline defined above will be accessible as `iterator_output['labels']`,
 - `reader_name`: setting this parameter introduces the notion of an epoch to our iterator. DALI pipeline itself is infinite, it will return the data indefinately, wrapping around the dataset. DALI readers (such as `fn.readers.caffe2` used in this example) have access to the information about the size of the dataset. If we want to pass this information to the iterator we need to point to the operator that should be queried for the dataset size. We do it by naming the operator (note `name="mnist_caffe2_reader"`) and passing the same name as the value for `reader_name` argument,
  - `auto_reset`: this argument controls the behaviour of the iterator after the end of an epoch. If set to `True` will automatically reset the state of the iterator and prepare it to start the next epoch.

In [4]:
from nvidia.dali.plugin import jax as dax


print('Creating iterators')
training_iterator = dax.DALIGenericIterator(
    training_pipeline,
    output_map=["images", "labels"],
    reader_name="mnist_caffe2_reader",
    auto_reset=True)

validation_iterator = dax.DALIGenericIterator(
    validation_pipeline,
    output_map=["images", "labels"],
    reader_name="mnist_caffe2_reader",
    auto_reset=True)

print(training_iterator)
print(f"Training iterator size = {training_iterator.size}")
print(f"Validation iterator size = {validation_iterator.size}")

Creating iterators
Training iterator size = 60000
Validation iterator size = 10000


With the setup above DALI iterators are ready for the training. 

Next we import training utilities implemented in JAX. `init_model` will create the model instance and initialize its parameters. In this simple example it is a MLP model with two hidden layers. `update` performs one iteration of the training. `accuracy` is a helper function to run validation after each epoch on the test set and get current accuracy of the model.

In [5]:
from model import init_model, update, accuracy

At this point everything is ready to run the training.

In [6]:
print('Starting training')

model = init_model()
num_epochs = 10

for epoch in range(num_epochs):
    for batch in training_iterator:
        model = update(model, batch)

    test_acc = accuracy(model, validation_iterator)
    print(f"Epoch {epoch} sec")
    print(f"Test set accuracy {test_acc}")

Starting training
Epoch 0 sec
Test set accuracy 0.784600019454956
Epoch 1 sec
Test set accuracy 0.8463000655174255
Epoch 2 sec
Test set accuracy 0.87090003490448
Epoch 3 sec
Test set accuracy 0.8835000395774841
Epoch 4 sec
Test set accuracy 0.891800045967102
Epoch 5 sec
Test set accuracy 0.8980000615119934
Epoch 6 sec
Test set accuracy 0.9033000469207764
Epoch 7 sec
Test set accuracy 0.9085000157356262
Epoch 8 sec
Test set accuracy 0.9119000434875488
Epoch 9 sec
Test set accuracy 0.9140000343322754


### Multiple GPUs

This section shows how to train the network with multiple GPUs. This simple example discusses setup with many GPUs availible in one JAX process on a single node.

We start by modifiying pipeline definition function. 

Note new arguments passed to the `fn.readers.caffe2`: `num_shards` and `shard_id`. They are used to controll sharding:
 - `num_shards` sets total number of shards
 - `shard_id` tells the pipeline for which shard in the training it is responsible. 

 Also `device_id` argument was removed from the decorator. Since we want these pipelines run on different GPUs we will pass particualr `device_id` in pipeline creation. Most often, `device_id` and `shard_id` will have the same value but it is not a requirement.

 If you want to learn more about DALI sharding behaviour look into [DALI sharding docs page]().

In [7]:
num_devices = 2

batch_size_per_gpu = batch_size // num_devices


In [8]:

@pipeline_def(batch_size=batch_size_per_gpu, num_threads=4)
def mnist_sharded_pipeline(data_path, random_shuffle, num_shards, shard_id):
    jpegs, labels = fn.readers.caffe2(
        path=data_path,
        random_shuffle=random_shuffle,
        name="mnist_caffe2_reader",
        num_shards=num_shards,
        shard_id=shard_id)
    images = fn.decoders.image(
        jpegs, device='mixed', output_type=types.GRAY)
    images = fn.crop_mirror_normalize(
        images, dtype=types.FLOAT, std=[255.], output_layout="CHW")
    images = fn.reshape(images, shape=[image_size * image_size])

    labels = labels.gpu()
    
    if random_shuffle:
        labels = fn.one_hot(labels, num_classes=num_classes)

    return images, labels

Creating pipelines. Note the `device_id` values that are passed to place a pipeline on a different device.

In [9]:
print('Creating training pipelines')
training_pipeline_0 = mnist_sharded_pipeline(data_path=training_data_path, random_shuffle=True, num_shards=2, shard_id=0, device_id=0)
training_pipeline_1 = mnist_sharded_pipeline(data_path=training_data_path, random_shuffle=True, num_shards=2, shard_id=1, device_id=1)

print(f'Pipeline {training_pipeline_0} working on device {training_pipeline_0.device_id}')
print(f'Pipeline {training_pipeline_1} working on device {training_pipeline_1.device_id}')

validation_pipeline = mnist_sharded_pipeline(data_path=validation_data_path, random_shuffle=False, num_shards=1, shard_id=0, device_id=0)

Creating training pipelines
Pipeline <nvidia.dali.pipeline.Pipeline object at 0x7d9b7c0aa350> working on device 0
Pipeline <nvidia.dali.pipeline.Pipeline object at 0x7d9b7c0a9ab0> working on device 1


In [10]:
import jax.numpy as jnp
from jax import tree_map


model = init_model()
model = tree_map(lambda x: jnp.array([x] * num_devices), model) 

In [11]:
from model import update_parallel
from jax import pmap

update_parallel = pmap(update_parallel, axis_name='data')

In [12]:
print('Creating training iterator')

training_iterator = dax.DALIGenericIterator(
    [training_pipeline_0, training_pipeline_1],
    output_map=["images", "labels"],
    reader_name="mnist_caffe2_reader",
    auto_reset=True)

validation_iterator = dax.DALIGenericIterator(
    validation_pipeline,
    output_map=["images", "labels"],
    reader_name="mnist_caffe2_reader",
    auto_reset=True)

print(f"Training iterator size = {training_iterator.size}")
print(f"Validation iterator size = {validation_iterator.size}")



Creating training iterator
Training iterator size = 30000
Validation iterator size = 10000


In [13]:
for epoch in range(num_epochs):
    for batch in training_iterator:
        model = update_parallel(model, batch)
        
    test_acc = accuracy(tree_map(lambda x: x[0], model), validation_iterator)
    
    print(f"Epoch {epoch} sec")
    print(f"Test set accuracy {test_acc}")


Epoch 0 sec
Test set accuracy 0.7860000133514404
Epoch 1 sec
Test set accuracy 0.8438000679016113
Epoch 2 sec
Test set accuracy 0.8681000471115112
Epoch 3 sec
Test set accuracy 0.8776000142097473
Epoch 4 sec
Test set accuracy 0.8905000686645508
Epoch 5 sec
Test set accuracy 0.8975000381469727
Epoch 6 sec
Test set accuracy 0.9029000401496887
Epoch 7 sec
Test set accuracy 0.9031000137329102
Epoch 8 sec
Test set accuracy 0.9109000563621521
Epoch 9 sec
Test set accuracy 0.9129000306129456
